## Задание

- Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
- Cохранить результаты в pandas dataframe

---

Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных:

- Привести колонки датафрейма к одному типу данных
- Проверить индекс, чтобы он соответствовал порядку строк
- Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки

---

Провести мини-исследование рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.

- Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены.
- Рассчитать объемы вводимого жилья по регионам и по годам.
- Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах
- Визуализировать разброс, связь и распределение этих показателей по регионам
- Сделать выводы по каждому из расчетов и графиков


### Загрузка библиотек


In [4]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.express as px

In [5]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 150
pd.options.display.precision = 2
pd.options.display.show_dimensions = True
pd.options.display.precision = 6


### Сбор информации


In [3]:
# Получение списка объектов

limit_ = 1000
all_obj_ids = []
for offset_ in range(0, 12000, 1000):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    response = requests.get(url)
    objects_data = response.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    all_obj_ids.extend(objids)


KeyboardInterrupt: 

In [ ]:
len(all_obj_ids)


In [ ]:
# Сбор информации по объектам

all_objects_description = []

for id in all_obj_ids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    response_object = requests.get(url)
    obj = response_object.json()
    all_objects_description.append(obj)


### Загрузка DataFrame


In [6]:
# Список используемых колонок

use_columns = [
    'data.id', 'data.region', 'data.objElemLivingCnt', 'data.objReady100PercDt', 'data.wallMaterialShortDesc',
    'data.objLkFinishTypeDesc', 'data.objLkClassDesc', 'data.objFloorCnt',
    'data.objFlatSq', 'data.objInfrstrBicycleLaneFlg', 'data.objInfrstrPlaygrndCnt', 'data.objInfrstrSportGroundCnt',
    'data.objInfrstrTrashAreaCnt', 'data.objInfrstrRampFlg',
    'data.objInfrstrCurbLoweringFlg', 'data.objElevatorWheelchairCnt',
    'data.objElevatorPassengerCnt', 'data.objElevatorCargoCnt',
    'data.objPriceAvg', 'data.nonlivFirstFloor', 'data.objEnergyEffShortDesc', 'data.infrastructureIndexValue'
]


In [7]:
# Словарь для переименования колонок
name_columns = {
    'data.id': 'Id_объекта',
    'data.developer.devShortCleanNm': 'Краткое наименование организации',
    'data.region': 'Регион строительства',
    'data.address': 'Адрес строительства',
    'data.objElemLivingCnt': 'Количество квартир',
    'data.objReady100PercDt': 'Ввод в эксплуатацию',
    'data.wallMaterialShortDesc': 'Материал стен',
    'data.objLkFinishTypeDesc': 'Тип отделки',
    'data.objLkFreePlanDesc': 'Свободная планировка',
    'data.objElemParkingCnt': 'Количество мест в паркинге',
    'data.objLkLatitude': 'Широта',
    'data.objLkLongitude': 'Долгота',
    'data.objLkClassDesc': 'Тип недвижимости',
    'data.objFloorCnt': 'Количество этажей',
    'data.objFlatCnt': 'Количество квартир',
    'data.objFlatSq': 'Жилая площадь, м²',
    'data.objNonlivElemCnt': 'Количество нежилых помещений',
    'data.objStatus': 'Статус строительства',
    'data.objTransferPlanDt': 'Выдача ключей',
    'data.objInfrstrBicycleLaneFlg': 'Велосипедные дорожки',
    'data.objInfrstrPlaygrndCnt': 'Количество детских площадок',
    'data.objInfrstrSportGroundCnt': 'Количество спортивных площадок',
    'data.objInfrstrTrashAreaCnt': 'Количество площадок для сбора мусора',
    'data.objInfrstrRampFlg': 'Наличие пандуса',
    'data.objInfrstrCurbLoweringFlg': 'Наличие понижающих площадок',
    'data.objElevatorWheelchairCnt': 'Количество инвалидных подъемников',
    'data.objElevatorPassengerCnt': 'Количество пассажирских лифтов',
    'data.objElevatorCargoCnt': 'Количество грузовых лифтов',
    'data.soldOutPerc': 'Распроданность квартир',
    'data.objPriceAvg': 'Средняя цена за 1 м²',
    'data.nonlivFirstFloor': 'Первый этаж нежилой',
    'data.objEnergyEffShortDesc': 'Класс энергоэффективности здания',
    'data.infrastructureIndexValue': 'Индекс доступности инфраструктуры',
    'data.quartography.objLivElem1KCnt': 'Количество 1-комнатных квартир',
    'data.quartography.objLivElem2KCnt': 'Количество 2-комнатных квартир',
    'data.quartography.objLivElem3KCnt': 'Количество 3-комнатных квартир',
    'data.quartography.objLivElem4KCnt': 'Количество 4-комнатных квартир',
    'data.quartography.objLivElemEntrCnt': 'Количество подъездов',
    'data.quartography.objLivElemEntrFloorAvg': 'Среднее количество квартир на этаже'
}


In [8]:
# словарь для переименования регионов строительства

regin_ict = {
    1: 'Республика Адыгея',
    2: 'Республика Башкортостан',
    3: 'Республика Бурятия',
    4: 'Республика Алтай (Горный Алтай)',
    5: 'Республика Дагестан',
    6: 'Республика Ингушетия',
    7: 'Кабардино-Балкарская Республика',
    8: 'Республика Калмыкия',
    9: 'Республика Карачаево-Черкессия',
    10: 'Республика Карелия',
    11: 'Республика Коми',
    12: 'Республика Марий Эл',
    13: 'Республика Мордовия',
    14: 'Республика Саха (Якутия)',
    15: 'Республика Северная Осетия — Алания',
    16: 'Республика Татарстан',
    17: 'Республика Тыва',
    18: 'Удмуртская Республика',
    19: 'Республика Хакасия',
    21: 'Чувашская Республика',
    22: 'Алтайский край',
    23: 'Краснодарский край',
    24: 'Красноярский край',
    25: 'Приморский край',
    26: 'Ставропольский край',
    27: 'Хабаровский край',
    28: 'Амурская область',
    29: 'Архангельская область',
    30: 'Астраханская область',
    31: 'Белгородская область',
    32: 'Брянская область',
    33: 'Владимирская область',
    34: 'Волгоградская область',
    35: 'Вологодская область',
    36: 'Воронежская область',
    37: 'Ивановская область',
    38: 'Иркутская область',
    39: 'Калининградская область',
    40: 'Калужская область',
    41: 'Камчатский край',
    42: 'Кемеровская область',
    43: 'Кировская область',
    44: 'Костромская область',
    45: 'Курганская область',
    46: 'Курская область',
    47: 'Ленинградская область',
    48: 'Липецкая область',
    49: 'Магаданская область',
    50: 'Московская область',
    51: 'Мурманская область',
    52: 'Нижегородская область',
    53: 'Новгородская область',
    54: 'Новосибирская область',
    55: 'Омская область',
    56: 'Оренбургская область',
    57: 'Орловская область',
    58: 'Пензенская область',
    59: 'Пермский край',
    60: 'Псковская область',
    61: 'Ростовская область',
    62: 'Рязанская область',
    63: 'Самарская область',
    64: 'Саратовская область',
    65: 'Сахалинская область',
    66: 'Свердловская область',
    67: 'Смоленская область',
    68: 'Тамбовская область',
    69: 'Тверская область',
    70: 'Томская область',
    71: 'Тульская область',
    72: 'Тюменская область',
    73: 'Ульяновская область',
    74: 'Челябинская область',
    75: 'Забайкальский край',
    76: 'Ярославская область',
    77: 'г. Москва',
    78: 'г. Санкт-Петербург',
    79: 'Еврейская автономная область',
    82: 'Республика Крым',
    83: 'Ненецкий автономный округ',
    86: 'Ханты-Мансийский автономный округ — Югра',
    87: 'Чукотский автономный округ',
    89: 'Ямало-Ненецкий автономный округ',
    92: 'г. Севастополь',
    95: 'Чеченская республика'
}


In [9]:
# data = pd.DataFrame.from_dict(pd.json_normalize(all_objects_description), orient='columns')

data = pd.read_csv('dom.csv', usecols=use_columns,
                   parse_dates=['data.objReady100PercDt'])


In [10]:
# переименовываем колонки

data.rename(columns=name_columns, inplace=True)


In [11]:
#  Меняем цифровое значение региона

data['Регион строительства'] = data['Регион строительства'].map(regin_ict)


In [12]:
data.head()


,Id_объекта,Регион строительства,Количество квартир,Ввод в эксплуатацию,Материал стен,Тип отделки,Тип недвижимости,"Жилая площадь, м²",Велосипедные дорожки,Наличие пандуса,Наличие понижающих площадок,Класс энергоэффективности здания,Индекс доступности инфраструктуры,Количество этажей,Количество детских площадок,Количество спортивных площадок,Количество площадок для сбора мусора,Количество инвалидных подъемников,Количество пассажирских лифтов,Количество грузовых лифтов,Средняя цена за 1 м²,Первый этаж нежилой
0,29679,Ростовская область,18,2020-03-31,Другое,Под ключ,Комфорт,1519.00,0,0,0,Не нормируется,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29678,Ростовская область,27,2020-03-31,Другое,Под чистовую,Комфорт,1519.00,0,0,0,Не нормируется,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13383,Тюменская область,68,2022-12-31,Другое,По желанию заказчика,Элитный,6293.78,0,1,1,B,10,20.0,1.0,1.0,1.0,0.0,5.0,1.0,53109.0,1.0
3,13397,Тюменская область,65,2024-12-31,Другое,Без отделки,Элитный,6248.51,0,1,1,B,10,12.0,1.0,1.0,1.0,0.0,5.0,4.0,45523.0,0.0
4,13399,Тюменская область,0,2024-12-31,Другое,Без отделки,Элитный,0.00,0,1,1,B,10,1.0,0.0,0.0,1.0,0.0,8.0,7.0,50768.0,1.0


In [13]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10527 entries, 0 to 10526
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Id_объекта                            10527 non-null  int64         
 1   Регион строительства                  10316 non-null  object        
 2   Количество квартир                    10527 non-null  int64         
 3   Ввод в эксплуатацию                   10527 non-null  datetime64[ns]
 4   Материал стен                         10527 non-null  object        
 5   Тип отделки                           10527 non-null  object        
 6   Тип недвижимости                      10527 non-null  object        
 7   Жилая площадь, м²                     10527 non-null  float64       
 8   Велосипедные дорожки                  10527 non-null  int64         
 9   Наличие пандуса                       10527 non-null  int64         
 10

In [14]:
#  Количество пропущенных значений

data.isna().sum().sort_values(ascending=False)


Средняя цена за 1 м²                    2906
Регион строительства                     211
Количество спортивных площадок            81
Количество грузовых лифтов                79
Количество пассажирских лифтов            79
Количество инвалидных подъемников         79
Количество площадок для сбора мусора      79
Количество детских площадок               79
Первый этаж нежилой                       11
Количество этажей                          2
Индекс доступности инфраструктуры          0
Id_объекта                                 0
Наличие понижающих площадок                0
Наличие пандуса                            0
Велосипедные дорожки                       0
Жилая площадь, м²                          0
Тип недвижимости                           0
Тип отделки                                0
Материал стен                              0
Ввод в эксплуатацию                        0
Количество квартир                         0
Класс энергоэффективности здания           0
Length: 22

In [15]:
# Количество пропущенных значений в %
(data.isna().sum()/data.shape[0]).round(2).sort_values(ascending=False)


Средняя цена за 1 м²                    0.28
Регион строительства                    0.02
Количество грузовых лифтов              0.01
Количество пассажирских лифтов          0.01
Количество инвалидных подъемников       0.01
Количество площадок для сбора мусора    0.01
Количество спортивных площадок          0.01
Количество детских площадок             0.01
Id_объекта                              0.00
Количество этажей                       0.00
Индекс доступности инфраструктуры       0.00
Класс энергоэффективности здания        0.00
Наличие понижающих площадок             0.00
Наличие пандуса                         0.00
Велосипедные дорожки                    0.00
Жилая площадь, м²                       0.00
Тип недвижимости                        0.00
Тип отделки                             0.00
Материал стен                           0.00
Ввод в эксплуатацию                     0.00
Количество квартир                      0.00
Первый этаж нежилой                     0.00
Length: 22

Наибольшее количество пропусков - 28% - в колонке "Средняя цена за 1 м²". Так как эта цифра привязана к конкретному дому, я не вижу способа её заполнить. Поэтому все наблюдения с пропущенными значениями я удаляю.


In [16]:
data_clean = data.dropna()
data_clean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7427 entries, 2 to 10526
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Id_объекта                            7427 non-null   int64         
 1   Регион строительства                  7427 non-null   object        
 2   Количество квартир                    7427 non-null   int64         
 3   Ввод в эксплуатацию                   7427 non-null   datetime64[ns]
 4   Материал стен                         7427 non-null   object        
 5   Тип отделки                           7427 non-null   object        
 6   Тип недвижимости                      7427 non-null   object        
 7   Жилая площадь, м²                     7427 non-null   float64       
 8   Велосипедные дорожки                  7427 non-null   int64         
 9   Наличие пандуса                       7427 non-null   int64         
 10 

In [17]:
# Список колонок для приведения числовому типу

column_to_int = ['Количество этажей',
                 'Количество детских площадок', 'Количество спортивных площадок',
                 'Количество площадок для сбора мусора',
                 'Количество инвалидных подъемников', 'Количество пассажирских лифтов',
                 'Количество грузовых лифтов', 'Первый этаж нежилой',
                 'Id_объекта', 'Количество квартир',
                 'Велосипедные дорожки', 'Наличие пандуса',
                 'Наличие понижающих площадок', 'Индекс доступности инфраструктуры']


In [18]:
# приведение к единому типу данных

data_clean[column_to_int] = data_clean[column_to_int].astype('int32')


/var/folders/rw/fy4jnmhn3l72v47ky8bt9r9w0000gn/T/ipykernel_10055/186447843.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean[column_to_int] = data_clean[column_to_int].astype('int32')


In [19]:
data_clean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7427 entries, 2 to 10526
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Id_объекта                            7427 non-null   int32         
 1   Регион строительства                  7427 non-null   object        
 2   Количество квартир                    7427 non-null   int32         
 3   Ввод в эксплуатацию                   7427 non-null   datetime64[ns]
 4   Материал стен                         7427 non-null   object        
 5   Тип отделки                           7427 non-null   object        
 6   Тип недвижимости                      7427 non-null   object        
 7   Жилая площадь, м²                     7427 non-null   float64       
 8   Велосипедные дорожки                  7427 non-null   int32         
 9   Наличие пандуса                       7427 non-null   int32         
 10 

In [20]:
# Создадим колонку с годом ввода в эксплуатацию

data_clean['Год ввода в эксплуатацию'] = data_clean['Ввод в эксплуатацию'].dt.year


/var/folders/rw/fy4jnmhn3l72v47ky8bt9r9w0000gn/T/ipykernel_10055/3825119810.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Год ввода в эксплуатацию'] = data_clean['Ввод в эксплуатацию'].dt.year


### Исследование

In [21]:
def procent(a,b):
    x = a/len(b)*100
    return(print(x, '%'))

In [22]:
#  Количество объектов строительства по регионам

data_clean.groupby('Регион строительства')['Id_объекта'].count().sort_values(ascending=False)

Регион строительства
г. Москва                                   687
Краснодарский край                          653
Московская область                          574
г. Санкт-Петербург                          473
Новосибирская область                       285
Свердловская область                        247
Ленинградская область                       231
Тюменская область                           206
Ростовская область                          199
Республика Татарстан                        192
Республика Башкортостан                     191
Калининградская область                     164
Приморский край                             161
Иркутская область                           137
Самарская область                           134
Воронежская область                         122
Красноярский край                           119
Челябинская область                         113
Нижегородская область                       112
Удмуртская Республика                       109
Пермский край      

#### Цена за квадратный номер

In [23]:
fig = px.box(data_clean, x='Средняя цена за 1 м²', y='Регион строительства', color='Тип недвижимости', width=1200, height=3000, )
fig.show()

In [24]:
data_mean = data_clean.groupby('Регион строительства').mean()
avg_prise_region = data_mean['Средняя цена за 1 м²'].sort_values(ascending=False)
print('5 регионов с максимальными значениями средней стоимости квадратного метра')
print(avg_prise_region.head())
print('---')
print('5 регионов с минимальными значениями средней стоимости квадратного метра')
print(avg_prise_region.tail())
print('---')

5 регионов с максимальными значениями средней стоимости квадратного метра
Регион строительства
г. Москва              308954.768559
г. Санкт-Петербург     166280.319239
Сахалинская область    137465.088235
Московская область     121477.639373
Приморский край        117392.298137
Name: Средняя цена за 1 м², Length: 5, dtype: float64
---
5 регионов с минимальными значениями средней стоимости квадратного метра
Регион строительства
Кабардино-Балкарская Республика        46655.222222
Республика Северная Осетия — Алания    43459.454545
Костромская область                    42193.615385
Республика Дагестан                    24491.500000
Мурманская область                     20442.000000
Name: Средняя цена за 1 м², Length: 5, dtype: float64
---


In [25]:
data_mean.reset_index(inplace=True)

In [26]:
data_mean = data_mean.sort_values(by='Средняя цена за 1 м²', ascending=False)

In [27]:

fig = px.bar(data_mean, 
             x='Средняя цена за 1 м²',
             y='Регион строительства',
             log_x=True,
             title='Цена за квадратный номер',
             width=800, height=1600)
fig.show()

In [28]:
# Самый дорогой квадратный метр
data_mean.loc[data_mean['Средняя цена за 1 м²'].idxmax()]

Регион строительства                        г. Москва
Id_объекта                               37012.219796
Количество квартир                         411.157205
Жилая площадь, м²                        21988.406201
Велосипедные дорожки                         0.219796
Наличие пандуса                              0.404658
Наличие понижающих площадок                   0.86754
Индекс доступности инфраструктуры            7.340611
Количество этажей                           22.321689
Количество детских площадок                  2.030568
Количество спортивных площадок               1.219796
Количество площадок для сбора мусора         1.604076
Количество инвалидных подъемников             0.42067
Количество пассажирских лифтов               5.672489
Количество грузовых лифтов                   2.705968
Средняя цена за 1 м²                    308954.768559
Первый этаж нежилой                          0.804949
Год ввода в эксплуатацию                  2023.222707
Name: 74, Length: 18, dtype:

In [29]:
# Самый дешевый квадратный метр
data_mean.loc[data_mean['Средняя цена за 1 м²'].idxmin()]

Регион строительства                    Мурманская область
Id_объекта                                          6005.0
Количество квартир                                    15.0
Жилая площадь, м²                                    865.1
Велосипедные дорожки                                   0.0
Наличие пандуса                                        0.0
Наличие понижающих площадок                            1.0
Индекс доступности инфраструктуры                      1.0
Количество этажей                                      3.0
Количество детских площадок                            0.0
Количество спортивных площадок                         0.0
Количество площадок для сбора мусора                   0.0
Количество инвалидных подъемников                      0.0
Количество пассажирских лифтов                         0.0
Количество грузовых лифтов                             0.0
Средняя цена за 1 м²                               20442.0
Первый этаж нежилой                                    0

#### Объем вводимого жилой площади по регионам и годам

In [30]:
year = data_clean.groupby('Год ввода в эксплуатацию')['Жилая площадь, м²'].sum()
data_year = pd.DataFrame(year)
data_year.reset_index(inplace=True)

In [31]:
data_year.sort_values(by='Жилая площадь, м²', ascending=False)

,Год ввода в эксплуатацию,"Жилая площадь, м²"
5,2023,32553725.20
4,2022,20761545.83
6,2024,19084578.12
7,2025,6771371.27
8,2026,1669636.70
9,2027,420166.68
13,2031,118030.10
3,2021,101525.47
11,2029,98591.92
10,2028,75058.51


In [32]:
fig = px.bar(data_year, x='Год ввода в эксплуатацию', y='Жилая площадь, м²',log_y=True, title='Объем вводимого жилой площади по годам')
fig.show()

In [33]:
s_region = data_clean.groupby('Регион строительства')['Жилая площадь, м²'].sum()
data_s_region = pd.DataFrame(s_region)
data_s_region.reset_index(inplace=True)

In [34]:
data_s_region = data_s_region.sort_values(by='Жилая площадь, м²', ascending=False)

In [35]:
fig = px.bar(data_s_region, y='Регион строительства', x='Жилая площадь, м²',log_x=True, title='Объем вводимого жилой площади по регионам', width=800, height=1600)
fig.show()


####  Количество этажей и средняя этажность по регионам

In [36]:
floor_sum = data_clean.groupby('Регион строительства')['Количество этажей'].aggregate(['sum']).sort_values(by='sum',ascending=False)

In [37]:
data_floor_sum = pd.DataFrame(floor_sum).reset_index()

In [38]:
fig = px.bar(data_floor_sum, y='Регион строительства', x='sum',log_x=True, title='Количество этажей по регионам', labels={'sum': 'Количество строящихся этажей'}, width=800, height=1600)
fig.show()

In [39]:
floor_mean = data_clean.groupby('Регион строительства')['Количество этажей'].aggregate(['mean']).sort_values(by='mean',ascending=False)

In [40]:
data_floor_mean = pd.DataFrame(floor_mean).reset_index()

In [41]:
fig = px.bar(data_floor_mean, y='Регион строительства', x='mean',log_x=True, title='Средняя этажность строящихся зданий', labels={'mean': 'Средняя этажность зданий'}, width=800, height=1600)
fig.show()

In [42]:
fig = px.box(data_clean, x='Количество квартир', y='Тип недвижимости', color='Тип отделки', width=1200, height=1200)
fig.show()